In [ ]:
import sys
sys.path.insert(1, '..')
from cocoa.geo import GeoManager as gm

In [ ]:
help(gm)

In [ ]:
g=gm()

In [ ]:
g.get_list_db()

In [ ]:
g.get_list_output()

In [ ]:
g.get_list_standard()

In [ ]:
g.get_standard()

In [ ]:
g.set_standard('name')

In [ ]:
g.set_standard('oups') # the standard is not in the standard list 

In [ ]:
g.get_standard()

In [ ]:
g.to_standard('england')

In [ ]:
g.to_standard('European Union') # the name does not exist as a country

In [ ]:
g.to_standard('European Union',interpret_region=True) # interpretation of region, and output as dict

In [ ]:
g.to_standard('french') # difficult interpretation, getting first item

In [ ]:
g.to_standard(['spain'],output='pandas')